# EDA
---
1. 근로기간은 필요없다? 확인들어갑니다.
    - 삭제가 맞다.
2. 총계좌수가 의미가 있나?
    - 의미 없는것 같음.
    - 연체 계좌수 2이상은 날림.
3.
4.
5.
6.


In [ ]:
!pip3 install -U scikit-learn

In [ ]:
import os
import sys
__file__ = "./EDA_sb.ipynb"
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))

In [ ]:
from data.load import Data
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
train_data = Data.train
test_data = Data.test
submit_data = Data.submission


In [ ]:
# 한글 폰트 설정.
plt.rcParams['font.family'] = 'NanumGothic'
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

In [ ]:
oject_dict

In [ ]:
# 기존의 간단한 전처리
train_data = Data.train
test_data = Data.test
submit_data = Data.submission
# 상환개월 신규 피처 생성.
train_data['대출기간'] = train_data['대출기간'].astype(str).str[:3].astype(int)
train_data['월원금상환금'] = train_data['대출금액']/train_data['대출기간']
train_data['상환개월'] = train_data['총상환원금']/train_data['월원금상환금']

train_data.drop(['월원금상환금'],axis=1, inplace=True)
train_data['상환개월'] = train_data['상환개월'].round().astype(int)

# 주택 소유상태 Any 데이터 제거(9만6천개중 단ㄱ1개)
train_data[train_data["주택소유상태"] != "ANY"].reset_index(drop = True,inplace = True)

# 대출 목적 라벨링.
train_object = train_data['대출목적'].unique()
oject_dict = dict()
for i, v in enumerate(train_data['대출목적'].unique()):
    oject_dict[v] = i
# 만약에 없다면 위의 데이터에 없다면 기타로 설정.
train_data['대출목적'] = train_data['대출목적'].apply(lambda x: oject_dict.get(x, '기타'))

# 근로기간 라벨링.
train_data['근로기간'] = train_data['근로기간'].astype(str)
train_data['근로기간'] = train_data['근로기간'].str.extract('(\d+)').astype(float)  # 숫자만 추출
# 'Unknown'은 NaN으로 변환되므로, 이를 -1로 대체
train_data['근로기간'].fillna(-1, inplace=True)
# '10+' ->  11로 대체
train_data.loc[train_data['근로기간'] == 10, '근로기간'] = 11
# '<1 year'  -> 0로 대체
train_data.loc[train_data['근로기간'].isna(), '근로기간'] = 0

# 대출 등급, 주택소유상태, 대출기간 라벨링.
le = LabelEncoder()
train_data['주택소유상태'] = le.fit_transform(train_data['주택소유상태'])
train_data['대출기간'] = le.fit_transform(train_data['대출기간'])
train_data['대출등급'] = le.fit_transform(train_data['대출등급'])

train_data.head(3)


In [ ]:
correlation =  train_data[['대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수',
       '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급',
       '상환개월']].corr()
correlation

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(data = correlation, annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

# 근로기간 

In [ ]:
# 비율로 살펴보기
df =  train_data[['근로기간', '대출등급']]
# 각 근로기간별 대출 등급의 개수
df['등급별_건수'] = df.groupby(['근로기간', '대출등급'])['대출등급'].transform('count')

# 각 근로기간의 총 대출 건수
df['총_건수'] = df.groupby('근로기간')['대출등급'].transform('count')

# 비율 계산
df['비율'] = df['등급별_건수'] / df['총_건수']

plt.figure(figsize=(10,6))
sns.barplot(x='근로기간', y='비율', hue='대출등급', data=df)
plt.show()

In [ ]:
le = LabelEncoder()
train_data['대출등급'] = le.fit_transform(train_data['대출등급'])

In [ ]:
corr1 = train_data[['근로기간', '대출등급']].corr()
corr1

In [ ]:
# 근로기간 제거
train_data.drop(['근로기간'], axis=1, inplace=True)

# 총 계좌 수


In [ ]:
df = train_data[['대출등급', '총계좌수']]
plt.figure(figsize=(40,6))
sns.countplot(x='총계좌수', hue='대출등급', data=df)
plt.xticks(range(int(min(df['총계좌수'])), 51))
plt.show()

In [ ]:
corr2 = train_data[['총계좌수', '대출등급']].corr()
corr2

In [ ]:
corr3 = train_data[['연체계좌수', '대출등급']].corr()
corr3

In [ ]:
# 신규 피처로 연체계좌수 비율
train_data['연체계좌비율'] =  train_data['연체계좌수'] / train_data['총계좌수']
corr4 = train_data[['연체계좌비율', '대출등급']].corr()
corr4

In [ ]:
df = df[df['총계좌수'] < 50]
# 각 총계좌수별 대출 등급의 개수를 구합니다.
df['등급별_건수'] = df.groupby(['총계좌수', '대출등급'])['대출등급'].transform('count')

# 각 총계좌수의 총 대출 건수를 구합니다.
df['총_건수'] = df.groupby('총계좌수')['대출등급'].transform('count')

# 비율을 계산합니다.
df['비율'] = df['등급별_건수'] / df['총_건수']

# 비율을 시각화합니다.
plt.figure(figsize=(15,6))
sns.barplot(x='총계좌수', y='비율', hue='대출등급', data=df)
plt.show()

In [ ]:
df =train_data[['연체계좌수', '대출등급']]

# 각 총계좌수별 대출 등급의 개수를 구합니다.
df['등급별_건수'] = df.groupby(['연체계좌수', '대출등급'])['대출등급'].transform('count')

# 각 총계좌수의 총 대출 건수를 구합니다.
df['총_건수'] = df.groupby('연체계좌수')['대출등급'].transform('count')

# 비율을 계산합니다.
df['비율'] = df['등급별_건수'] / df['총_건수']

# 비율을 시각화합니다.
plt.figure(figsize=(15,6))
sns.barplot(x='연체계좌수', y='비율', hue='대출등급', data=df)
plt.show()

* 연체계좌수도 날리기

In [ ]:
train_data.columns

# 학습

In [ ]:
import numpy as np
import pickle   
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import optuna


In [ ]:
train_data = Data.train
test_data = Data.test
submit_data = Data.submission
# 기존의 간단한 전처리
# 상환개월 신규 피처 생성.
train_data['대출기간'] = train_data['대출기간'].astype(str).str[:3].astype(int)
train_data['월원금상환금'] = train_data['대출금액']/train_data['대출기간']
train_data['상환개월'] = train_data['총상환원금']/train_data['월원금상환금']
train_data.drop(['월원금상환금'],axis=1, inplace=True)
train_data['상환개월'] = train_data['상환개월'].round().astype(int)

# 주택 소유상태 Any 데이터 제거(9만6천개중 단ㄱ1개)
train_data[train_data["주택소유상태"] != "ANY"].reset_index(drop = True,inplace = True)

# 대출 목적 라벨링.
train_object = train_data['대출목적'].unique()
oject_dict = dict()
for i, v in enumerate(train_data['대출목적'].unique()):
    oject_dict[v] = i
# 만약에 없다면 위의 데이터에 없다면 기타로 설정.
train_data['대출목적'] = train_data['대출목적'].apply(lambda x: oject_dict.get(x, '기타'))

# 근로기간 라벨링.
train_data['근로기간'] = train_data['근로기간'].astype(str)
train_data['근로기간'] = train_data['근로기간'].str.extract('(\d+)').astype(float)  # 숫자만 추출
# 'Unknown'은 NaN으로 변환되므로, 이를 -1로 대체
train_data['근로기간'].fillna(-1, inplace=True)
# '10+' ->  11로 대체
train_data.loc[train_data['근로기간'] == 10, '근로기간'] = 11
# '<1 year'  -> 0로 대체
train_data.loc[train_data['근로기간'].isna(), '근로기간'] = 0

# 대출 등급, 주택소유상태, 대출기간 라벨링.
le = LabelEncoder()
train_data['주택소유상태'] = le.fit_transform(train_data['주택소유상태'])
train_data['대출기간'] = le.fit_transform(train_data['대출기간'])
train_data['대출등급'] = le.fit_transform(train_data['대출등급'])

train_data['연간소득'] = pd.qcut(train_data['연간소득'], q=10, labels = False)
train_data["총상환금"] = train_data["총상환원금"] + train_data["총상환이자"]
train_data["남은대출금액"] = train_data["대출금액"] - train_data["총상환금"]
train_data = train_data.drop(columns=['총상환금'])
train_data.head(3)


In [ ]:
# 근로기간 제거
train_data = train_data.drop(columns=['ID', '근로기간'])
train_data.head()

In [ ]:
train_data.corr()

In [ ]:
train_x = train_data.drop(columns=['대출등급'])
train_y = train_data['대출등급']

test_x = test_data.drop(columns=['ID', '근로기간'])
# 기존의 간단한 전처리
# 상환개월 신규 피처 생성.
test_x['대출기간'] = test_x['대출기간'].astype(str).str[:3].astype(int)
test_x['월원금상환금'] = test_x['대출금액']/test_x['대출기간']
test_x['상환개월'] = test_x['총상환원금']/test_x['월원금상환금']

test_x.drop(['월원금상환금'],axis=1, inplace=True)
test_x['상환개월'] = test_x['상환개월'].round().astype(int)

# 기타로 처리.
test_x['대출목적'] = test_x['대출목적'].apply(lambda x: oject_dict.get(x, 8))

# 대출 등급, 주택소유상태, 대출기간 라벨링.
le1 = LabelEncoder()
test_x['주택소유상태'] = le1.fit_transform(test_x['주택소유상태'])
test_x['대출기간'] = le1.fit_transform(test_x['대출기간'])
test_x['연간소득'] = pd.qcut(test_x['연간소득'], q=10, labels = False)
test_x["총상환금"] = test_x["총상환원금"] + test_x["총상환이자"]
test_x["남은대출금액"] = test_x["대출금액"] - test_x["총상환금"]
test_x = test_x.drop(columns=['총상환금'])
test_x.head(3)

In [ ]:
def objective(trial):
    # 하이퍼파라미터 탐색 공간 설정
    param = {
        'objective': 'multiclass',  
        'metric': 'multi_logloss',  
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_class': 7, 
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

    # 모델 생성 및 학습
    lgbm = lgb.LGBMClassifier(**param)
    early_stopping_callback = lgb.early_stopping(stopping_rounds=100, verbose=True)
    lgbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[early_stopping_callback])

    # 평가
    y_pred = lgbm.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

In [ ]:
import optuna
# 스터디 객체 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500, show_progress_bar=True)
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')
y_pred = lgbm.predict(val_features)
accuracy = accuracy_score(val_labels, y_pred)
print(f'Validation Accuracy: {accuracy:.2f}')

In [65]:
# 최적화된 하이퍼파라미터 적용
best_params = study.best_trial.params
lgbm = lgb.LGBMClassifier(**best_params)
lgbm.fit(train_x, train_y)  # 최종 모델 학습

In [66]:

pred = lgbm.predict(test_x)
pred = le.inverse_transform(pred)

[LightGBM] [Warning] feature_fraction is set=0.5817499384765732, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5817499384765732
[LightGBM] [Warning] lambda_l1 is set=0.026660975494242636, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.026660975494242636
[LightGBM] [Warning] lambda_l2 is set=7.980367474195268e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.980367474195268e-07
[LightGBM] [Warning] bagging_fraction is set=0.9985208349482491, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9985208349482491
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


In [68]:
submit_data['대출등급'] = pred
submit_data

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [69]:
submit_data.to_csv('./sample_submission.csv', index=False)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42)
model.fit(train_x, train_y)

In [ ]:
oject_dict

In [ ]:
# 기존의 간단한 전처리
# 상환개월 신규 피처 생성.
test_x['대출기간'] = test_x['대출기간'].astype(str).str[:3].astype(int)
test_x['월원금상환금'] = test_x['대출금액']/test_x['대출기간']
test_x['상환개월'] = test_x['총상환원금']/test_x['월원금상환금']

test_x.drop(['월원금상환금'],axis=1, inplace=True)
test_x['상환개월'] = test_x['상환개월'].round().astype(int)

# 기타로 처리.
test_x['대출목적'] = test_x['대출목적'].apply(lambda x: oject_dict.get(x, 8))

# 대출 등급, 주택소유상태, 대출기간 라벨링.
le1 = LabelEncoder()
test_x['주택소유상태'] = le1.fit_transform(test_x['주택소유상태'])
test_x['대출기간'] = le1.fit_transform(test_x['대출기간'])

test_x.head(3)


In [ ]:
test_x.info()

In [ ]:
pred = model.predict(test_x)
print(pred)
pred = le.inverse_transform(pred)
pred

In [ ]:
submit_data['대출등급'] = pred
submit_data

In [ ]:
submit_data.to_csv('sample_submission.csv', index=False)